In [1]:
import pandas as pd
import numpy as np
import psycopg2
import datetime as dt
from transformers import DataCollatorWithPadding
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import torch
import torch.nn as nn

D:\karpov_ml\2_ml\ml_lesson_22_final_project_recs\venv39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Загружаем данные

In [2]:
# Подгружаем данные из SQL

user_data_sql = pd.read_sql(
"""
SELECT * FROM public.user_data
""",             
con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
    "postgres.lab.karpov.courses:6432/startml"                                 
)


post_data_sql = pd.read_sql(
"""
SELECT * 
FROM public.post_text_df
""",             
con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
    "postgres.lab.karpov.courses:6432/startml"                                 
)


train_feed_data = pd.read_sql(
"""
SELECT user_id, post_id, target, timestamp
FROM public.feed_data
WHERE (feed_data.action = 'view') AND (timestamp < '2021-12-22')
ORDER BY RANDOM()
limit 1000000
""",             
con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
    "postgres.lab.karpov.courses:6432/startml"                                 
)


test_feed_data = pd.read_sql(
"""
SELECT user_id, post_id, target, timestamp
FROM public.feed_data
WHERE (feed_data.action = 'view') AND (timestamp > '2021-12-22')
ORDER BY RANDOM()
limit 200000
""",             
con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
    "postgres.lab.karpov.courses:6432/startml"                                 
)

In [272]:
train_feed_data = train_feed_data.sort_values(by='timestamp', ascending=True).drop('timestamp', axis=1)
test_feed_data = test_feed_data.sort_values(by='timestamp', ascending=True).drop('timestamp', axis=1)

In [3]:
post_data = post_data_sql.copy()
user_data = user_data_sql.copy()

# Выделение признаков для user_data

In [4]:
# Подгружаем некоторые статистики по юзерам из таблицы 'feed_data', посчитанные за тот же период, за который взят тренировочный датасет

user_features = pd.read_sql(
"""
SELECT user_id, COUNT(target) AS total_views, SUM(target) AS total_likes, MAX(DATE(timestamp)) - MIN(DATE(timestamp)) AS total_days
FROM public.feed_data
WHERE (feed_data.action = 'view') AND (timestamp < '2021-12-16')
GROUP BY user_id 
limit 200000
""",             
con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
    "postgres.lab.karpov.courses:6432/startml"                                 
)

In [5]:
# Создаем новые признаки

user_features['likes_freq'] = (user_features['total_likes'] / user_features['total_views']).round(2)
user_features['daily_views'] = (user_features['total_views'] / user_features['total_days']).round(2)

In [6]:
# Выделение бинарных признаков из колонок 'City' и 'Country'

user_data['russia'] = (user_data['country']=='Russia').astype(int)
user_data['moscow'] = (user_data['city']=='Moscow').astype(int)
user_data['saint_p'] = (user_data['city']=='Saint Petersburg').astype(int)

user_data = user_data.drop(['city', 'country'], axis=1)

In [7]:
# OHE над категориальными колонками

categorical = ['exp_group', 'os', 'source']

for col in categorical:

    user_data[col] = user_data[col].astype(object)
    
    ohe_cols = pd.get_dummies(user_data[col], prefix=col, drop_first=True).astype(int)
    user_data = pd.concat([user_data.drop(col, axis=1), ohe_cols], axis=1)

D:\karpov_ml\2_ml\ml_lesson_22_final_project_recs\venv39\lib\site-packages\pandas\core\algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)


In [8]:
# Объединение датафреймов 
user_data = user_data.merge(user_features[['user_id', 'likes_freq', 'daily_views']], on='user_id')

In [9]:
user_data.head(3)

,user_id,gender,age,russia,moscow,saint_p,exp_group_1,exp_group_2,exp_group_3,exp_group_4,os_iOS,source_organic,likes_freq,daily_views
0,200,1,34,1,0,0,0,0,1,0,0,0,0.12,4.26
1,201,0,37,1,0,0,0,0,0,0,0,0,0.08,9.31
2,202,1,17,1,0,0,0,0,0,1,0,0,0.14,7.17


# Выделение признаков для post_data

In [10]:
# Подгружаем некоторые статистики по юзерам из таблицы 'feed_data', посчитанные за тот же период, за который взят тренировочный датасет

post_features = pd.read_sql(
"""
SELECT post_id, COUNT(target) AS total_views, SUM(target) AS total_likes, MAX(DATE(timestamp)) - MIN(DATE(timestamp)) AS days_old
FROM public.feed_data
WHERE (feed_data.action = 'view') AND (timestamp < '2021-12-16')
GROUP BY post_id 
limit 10000
""",             
con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
    "postgres.lab.karpov.courses:6432/startml"                                 
)

In [11]:
# В данных есть информация только о 6831 постах, при этом они все были впервые просмотрены в один день (даже если взять данные за всё время)
post_features.value_counts('days_old')

days_old
75    6831
dtype: int64

In [12]:
# Выделение новых признаков для постов

post_features['post_likes_freq'] = (post_features['total_likes'] / post_features['total_views']).round(3)
post_features['post_daily_views'] = (post_features['total_views'] / post_features['days_old']).round(3)

In [13]:
post_features = post_features[['post_id', 'post_likes_freq', 'post_daily_views']]

In [14]:
post_data = post_data.merge(post_features, on='post_id', how='outer')

In [15]:
for topic in post_data['topic']:
    post_data[post_data['topic']==topic] = post_data[post_data['topic']==topic].fillna(post_data[post_data['topic']==topic]['post_daily_views'].mean())

In [16]:
#Применим tf-idf для колонки 'text'

from sklearn.feature_extraction.text import TfidfVectorizer

pd.options.mode.chained_assignment = None

tfidf = TfidfVectorizer()
tfidf.fit(post_data['text'])

post_data['tfidf'] = 0.0
for i in range(post_data.shape[0]):
    post_data['tfidf'][i] = tfidf.transform([post_data['text'][i]]).T.todense().max().round(3)

In [17]:
# Создание признака "количество символов в тексте поста"

post_data['text_lenght'] = post_data['text']

for i in range(post_data.shape[0]):
    post_data['text_lenght'][i] = len(post_data['text'][i])

In [18]:
# OHE над колонкой 'topic' и удаление ненужных колонок

post_data['topic'] = post_data['topic'].astype(object)
    
ohe_cols = pd.get_dummies(post_data['topic'], prefix=col, drop_first=True).astype(int)
post_data = pd.concat([post_data.drop(['topic', 'text'], axis=1), ohe_cols], axis=1)

## Создаем обучающие и тестовые выборки

In [298]:
train_data = train_feed_data.merge(user_data, on='user_id').merge(post_data, on='post_id')
test_data = test_feed_data.merge(user_data, on='user_id').merge(post_data, on='post_id')

In [299]:
X_train = train_data.drop(['target', 'user_id', 'post_id'], axis=1)
X_test = test_data.drop(['target', 'user_id', 'post_id'], axis=1)

y_train = train_data['target']
y_test = test_data['target']

In [300]:
### Расчитываем баданс классов для таргета

from sklearn.utils.class_weight import compute_class_weight

class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)

## CatBoost

In [301]:
from catboost import CatBoostClassifier, Pool

control_model = CatBoostClassifier(class_weights=class_weights)


control_model.fit(X_train,
             y_train,
             )

Learning rate set to 0.196759
0:	learn: 0.6655441	total: 109ms	remaining: 1m 48s
1:	learn: 0.6469839	total: 218ms	remaining: 1m 48s
2:	learn: 0.6343774	total: 342ms	remaining: 1m 53s
3:	learn: 0.6284457	total: 451ms	remaining: 1m 52s
4:	learn: 0.6251048	total: 548ms	remaining: 1m 49s
5:	learn: 0.6197986	total: 659ms	remaining: 1m 49s
6:	learn: 0.6181852	total: 757ms	remaining: 1m 47s
7:	learn: 0.6161439	total: 855ms	remaining: 1m 46s
8:	learn: 0.6136788	total: 960ms	remaining: 1m 45s
9:	learn: 0.6129694	total: 1.07s	remaining: 1m 45s
10:	learn: 0.6114361	total: 1.18s	remaining: 1m 46s
11:	learn: 0.6101148	total: 1.28s	remaining: 1m 45s
12:	learn: 0.6091643	total: 1.38s	remaining: 1m 44s
13:	learn: 0.6083108	total: 1.49s	remaining: 1m 44s
14:	learn: 0.6075548	total: 1.6s	remaining: 1m 44s
15:	learn: 0.6070170	total: 1.69s	remaining: 1m 43s
16:	learn: 0.6068020	total: 1.79s	remaining: 1m 43s
17:	learn: 0.6066639	total: 1.95s	remaining: 1m 46s
18:	learn: 0.6064881	total: 2.06s	remaining: 

In [302]:
y_pred_cb = control_model.predict(X_test)

from sklearn.metrics import accuracy_score

print('Accuracy на тесте : {0:0.4f}'. format(accuracy_score(y_test, y_pred_cb)))

Accuracy на тесте : 0.6066


## Создаем новые признаки с помощью ембеддингов текстов постов

In [19]:
### Импортируем модель "Bert" для создания ембеддингов из постов

from transformers import AutoTokenizer, BertModel

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
model = BertModel.from_pretrained("bert-base-cased")

D:\karpov_ml\2_ml\ml_lesson_22_final_project_recs\venv39\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [20]:
### Воспользуемся вспомогательными классами Dataset и Dataloader из torch.utils

class CustomDataset(Dataset):
    
    def __init__(self, X):
        self.text = X

    def tokenize(self, text):
        return tokenizer(text, return_tensors='pt', padding='max_length', truncation=True, max_length=150)

    def __len__(self):
        return self.text.shape[0]

    def __getitem__(self, index):
        output = self.text[index]
        output = self.tokenize(output)
        return {k: v.reshape(-1) for k, v in output.items()}


eval_ds = CustomDataset(post_data_sql['text'])
eval_dataloader = DataLoader(eval_ds, batch_size=10)

In [21]:
### Для получения эмбеддингов, воспользуемся следующей функцией, в ней берется выход из последнего слоя и усредняется по каждому значению

def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output['last_hidden_state']
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

In [22]:
### Переводим модель в состояние валидации и отключаем подсчет градиента

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
model.eval()

embeddings = torch.Tensor().to(device)

with torch.no_grad():
    for n_batch, batch in enumerate(tqdm(eval_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        embeddings = torch.cat([embeddings, mean_pooling(outputs, batch['attention_mask'])])
    embeddings = embeddings.cpu().numpy()

100%|████████████████████████████████████████████████████████████████████████████████| 703/703 [15:23<00:00,  1.31s/it]


In [23]:
### Уменьшим размерность эмбеддингов с 768 до 15 с помощью метода главных компонент

from sklearn.decomposition import PCA

pca = PCA(n_components=15, random_state=42)
emb_15d = pca.fit_transform(embeddings)

In [24]:
### Запускаем алгоритм кластеризации

from sklearn.cluster import AgglomerativeClustering

clustering = AgglomerativeClustering(n_clusters=None, distance_threshold=0.5, affinity='cosine', linkage='average').fit(emb_15d)

In [25]:
pca = PCA(n_components=2, random_state=42)
emb_2d = pd.DataFrame(pca.fit_transform(embeddings), columns=['x1', 'x2'])
emb_2d['label'] = clustering.labels_
emb_2d['label'].nunique() # 16

39

In [26]:
def show_examples(cluster, n):
    for i in range(n):
        print(i, post_data_sql[emb_2d['label'] == cluster].reset_index()['text'][i].split('.')[0])
show_examples(cluster=1, n=6)

0 #SouravGanguly has tested negative for #COVID19
1 This is the highest number of #COVID19 tests conducted in the country in a single day so far
2 #COVID19 cumulative cases have reached 25 million
3 If a Non-Binary with a short neon blue hair cant have a hair cut due to #COVID19, are they/them still a valid Non-Binary?
4 The new case is a woman from Victoria in her 20s who had moved to SA for work
5 Before #COVID19 people were worried about black lung from vaping


In [27]:
### Добавляем фичу "label" в данные о постах

post_data['label'] = emb_2d['label']

In [28]:
### Применяем OHE к колонке "label"

post_data['label'] = post_data['label'].astype(object)
    
ohe_cols = pd.get_dummies(post_data['label'], prefix=col, drop_first=True).astype(int)
post_data = pd.concat([post_data.drop(['label'], axis=1), ohe_cols], axis=1)

D:\karpov_ml\2_ml\ml_lesson_22_final_project_recs\venv39\lib\site-packages\pandas\core\algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)


## Создаем train/test с новой фичей

In [305]:
train_data = train_feed_data.merge(user_data, on='user_id').merge(post_data, on='post_id')
test_data = test_feed_data.merge(user_data, on='user_id').merge(post_data, on='post_id')

In [306]:
X_train = train_data.drop(['target', 'user_id', 'post_id'], axis=1)
X_test = test_data.drop(['target', 'user_id', 'post_id'], axis=1)

y_train = train_data['target']
y_test = test_data['target']

## Обучаем CatBoost на новых данных

In [307]:
from catboost import CatBoostClassifier, Pool

test_model = CatBoostClassifier(class_weights=class_weights)


test_model.fit(X_train,
             y_train,
             )

Learning rate set to 0.196759
0:	learn: 0.6635569	total: 118ms	remaining: 1m 57s
1:	learn: 0.6447609	total: 242ms	remaining: 2m
2:	learn: 0.6344531	total: 349ms	remaining: 1m 56s
3:	learn: 0.6293835	total: 458ms	remaining: 1m 54s
4:	learn: 0.6234885	total: 575ms	remaining: 1m 54s
5:	learn: 0.6204423	total: 690ms	remaining: 1m 54s
6:	learn: 0.6173115	total: 803ms	remaining: 1m 53s
7:	learn: 0.6149029	total: 922ms	remaining: 1m 54s
8:	learn: 0.6141711	total: 1.02s	remaining: 1m 52s
9:	learn: 0.6136719	total: 1.13s	remaining: 1m 51s
10:	learn: 0.6117123	total: 1.24s	remaining: 1m 51s
11:	learn: 0.6107438	total: 1.36s	remaining: 1m 51s
12:	learn: 0.6099840	total: 1.47s	remaining: 1m 51s
13:	learn: 0.6090205	total: 1.58s	remaining: 1m 51s
14:	learn: 0.6085436	total: 1.71s	remaining: 1m 52s
15:	learn: 0.6077201	total: 1.85s	remaining: 1m 53s
16:	learn: 0.6071007	total: 1.96s	remaining: 1m 53s
17:	learn: 0.6065943	total: 2.07s	remaining: 1m 52s
18:	learn: 0.6064758	total: 2.18s	remaining: 1m 

In [308]:
y_pred_cb = test_model.predict(X_test)

from sklearn.metrics import accuracy_score

print('Accuracy на тесте : {0:0.4f}'. format(accuracy_score(y_test, y_pred_cb)))

Accuracy на тесте : 0.6082


## Сохраняем модели

In [309]:
control_model.save_model('control_model', format="cbm")
test_model.save_model('test_model', format="cbm")

In [29]:
# Записываем таблицу с user_data в sql 

from sqlalchemy import create_engine

engine = create_engine(
    "postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
    "postgres.lab.karpov.courses:6432/startml"
)

user_data.to_sql('n_habenko_14_users_lesson_22', con=engine, if_exists='replace')

205

In [30]:
# Записываем таблицу с post_data в sql 

from sqlalchemy import create_engine

engine = create_engine(
    "postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
    "postgres.lab.karpov.courses:6432/startml"
)

post_data.to_sql('n_habenko_14_posts_lesson_22', con=engine, if_exists='replace')

23

In [ ]:
data = user_data[user_data['user_id'] == 200].merge(post_data, how='cross').drop(['user_id', 'post_id'], axis=1)
predictions = pd.DataFrame()
predictions['predict_proba'] = catboost.predict_proba(data)[:, 1]
predictions['post_id'] = post_data_sql['post_id']
predictions['text'] = post_data_sql['text']
predictions['topic'] = post_data_sql['topic']
result = predictions.sort_values(by='predict_proba', ascending=False).head(5)

In [ ]:
result